In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

import sqlite3
import re

## Парсинг allthelyrics.com

In [2]:
# функция очистки текста от лишних символов
def replace_all(text, to_replace):
    for symbol in to_replace:
        text = text.replace(symbol, ' ')
    return text

In [3]:
# ссылки на исполнителей
urls = ['https://www.allthelyrics.com/lyrics/perfect_circle', 'https://www.allthelyrics.com/lyrics/pearl_jam',
       'https://www.allthelyrics.com/lyrics/tool', 'https://www.allthelyrics.com/lyrics/nine_inch_nails',
       'https://www.allthelyrics.com/lyrics/soundgarden', 'https://www.allthelyrics.com/lyrics/audioslave',
        'https://www.allthelyrics.com/lyrics/alice_in_chains']

# создадим список с ссылками на тексты
songs_lists = []

for url in urls:
    songs_list = requests.get(url)
    songs_soup = BeautifulSoup(songs_list.content, 'html.parser')

    for line in songs_soup.find_all('a'):  # вытащим из html адреса с текстами и добавим их в список
        if str(line).find('/lyrics/') != -1:
            song_url = "https://www.allthelyrics.com/" + line['href']
            songs_lists.append(song_url)          

songs_lists = list(set(songs_lists)) # удалим дубли
print(len(songs_lists))

658


In [21]:
# список с символами для удаления из текстов
symbols_to_del = ['!', '?', '.', ',', ':', '-', '"', '—', '  ', '–', '\r', 'Submitted by Guest', '|', '\xa0', '(', ')', '2x', '3x', 'x2', '[', ']', '4x', 'x6', 'x4', '*', '_', '\n']

In [22]:
parsed_songs = {}

for song in songs_lists:
    song_url = requests.get(song)
    song_soup = BeautifulSoup(song_url.content, 'html.parser')
    
    song_text = [string for string in song_soup.strings if len(string) > 2 and '\n' in string]
    text = ''.join(song_text)
    text = replace_all(text, symbols_to_del) # очищаем тексты от лишних символов  
    parsed_songs[song] = text
    time.sleep(2) # создает интервал между действиями, чтобы нас не забанили

In [23]:
len(parsed_songs)

658

In [24]:
parsed_songs_list = [] # создаем список с именами исполнителей и текстами
for key, value in parsed_songs.items():
    temp = [key,value]
    parsed_songs_list.append(temp)

In [25]:
parsed_songs_list[3:6]

[['https://www.allthelyrics.com//lyrics/perfect_circle/fiddle_and_the_drum-lyrics-489621.html',
  " My dear Johnny my dear friend And so once again you are fightin' us all And when I ask you why You raise your sticks and cry and I fall Oh my friend How did you come To trade the fiddle for the drum You say I have turned Like the enemies you've earned But I can remember All the good things you are And so I ask you please Can I help you find the peace and the star Oh my friend What time is this To trade the handshake for the fist And so once again Oh America my friend And so once again You are fighting us all And when we ask you why You raise your sticks and cry and we fall Oh my friend How did you come To trade the fiddle for the drum Like the enemies you've earned But we can remember All the good things you are And so we ask you please Can we help you find the peace and the star Oh my friend We have all come To fear the beating of your drum                                               

In [26]:
# подготовим тексты 
text_prep_all = []
artist_name = []
for i in range(len(parsed_songs_list)):
    text_prep = re.findall('[А-ЯA-Z][^А-ЯA-Z]*', parsed_songs_list[i][1]) # делим текст на строки
    text_prep = [a.lower() for a in text_prep] # убираем заглавные буквы
    text_prep = [a.rstrip().rstrip() for a in text_prep] # удаляем лишние пробелы
    text_prep = [a.lstrip() for a in text_prep]
    
    clean_text = [] # убираем короткие и пустые строки
    for val in text_prep:
        if len(val) <= 10:
            pass
        else:
            clean_text.append(val)
    
    if len(clean_text) < 4: # убираем короткие тексты
        pass
    else:
        text_prep_unique = list(set(clean_text)) # оставляем уникальные строки в текстах
        text_prep_str = '\n'.join(text_prep_unique) # добавляем символ табуляции
        text_prep_all.append(text_prep_str) # итоговый список с текстами
        artist_name.append(parsed_songs_list[i][0].split('/')[5]) # список с именами исполнителей

In [27]:
# создадим датафрейм и сохраним в csv
df_prepped = pd.DataFrame()


df_prepped['text'] = text_prep_all * 3
df_prepped['artist'] = artist_name * 3
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('~/GIT/Projects/ML-сервис/lyrics-generator/lyrics_3.csv', index=False)


print(df_prepped.shape)
df_prepped.head()

(1647, 2)


,artist,text
0,soundgarden,just a speck on the ground\ni'm born again\ni'...
1,perfect_circle,you raise your sticks and cry and\nhow did you...
2,pearl_jam,free of air and friction\nput me in a vacuum\n...
3,soundgarden,long time coming\nit seemed to satisfy\nmany s...
4,perfect_circle,i works on the levee mama both night and day\n...
